Lecture: AI I - Advanced 

Previous:
[**Chapter 2.3: Ensemble learning**](../03_ensemble.ipynb)

---

# Exercise 2.3: Ensemble learning

> Hint: When doing the exercises put your solution in the designated "Solution" section:
> ```python
> # Solution (put your code here)
> ```

## Task 1: Diabetes Regression with multiple Multi-Layer Perceptron

The diabetes dataset contains 442 samples with 10 baseline variables (age, sex, BMI, blood pressure, and 6 blood serum measurements). The target is a quantitative measure of disease progression one year after baseline.

**Tasks**
- Data Exploration & Understanding
- Data preparation
- Build Models: multiple Multi-Layer Perceptrons for regression (at least 3 with different architectures)
- Train the models 
- Construct an ensemble model from the individual models
- Evaluate the model performance using appropriate regression metrics (e.g. MSE, MAE) and Tensorboard


In [79]:
# prerequisites (don't edit this block)
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
x = diabetes.data
y = diabetes.target

In [80]:
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(42)  # set random seed for reproducibility

x_scaled = StandardScaler().fit_transform(x)
x_train, x_temp, y_train, y_temp = train_test_split(
    x_scaled, y, test_size=0.4, random_state=42
)
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42
)

batch_size = 16
train_loader = DataLoader(TensorDataset(torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32).view(-1,1)), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.tensor(x_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32).view(-1,1)), batch_size=batch_size)
test_loader = DataLoader(TensorDataset(torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32).view(-1,1)), batch_size=batch_size)

In [81]:
import torch.nn as nn
import torch.nn.functional as F

class FunnelNetwork(nn.Module):
    """
    Funnel architecture: Wide → Narrow
    Starts with many neurons and gradually reduces.
    Good for learning broad features then focusing.
    """
    def __init__(self, input_dim=10, output_dim=1):
        super().__init__()
        # 128 → 64 → 32 (funnel shape)
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, output_dim)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [82]:
class InvertedFunnelNetwork(nn.Module):
    """
    Inverted Funnel architecture: Narrow → Wide
    Starts focused and gradually expands representation.
    Builds increasingly complex features.
    """
    def __init__(self, input_dim=10, output_dim=1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, output_dim)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [83]:
class BrickNetwork(nn.Module):
    """
    Brick architecture: Constant width
    All hidden layers have the same number of neurons.
    Good for maintaining consistent representation.
    """
    def __init__(self, input_dim=10, output_dim=1):
        super().__init__()
        # 64 → 64 → 64 (brick/constant shape)
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, output_dim)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [84]:
def train_epoch(model, train_loader, criterion, optimizer):
    """Train for one epoch"""
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * batch_X.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_y).sum().item()
        total += batch_y.size(0)
    
    return train_loss / len(train_loader.dataset), correct / total


def evaluate(model, data_loader, criterion):
    """Evaluate the model"""
    model.eval()
    eval_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_X, batch_y in data_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            eval_loss += loss.item() * batch_X.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)
    
    return eval_loss / len(data_loader.dataset), correct / total


def train_model(model, train_loader, val_loader, num_epochs=150, lr=0.01):
    """Train a single model"""
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    
    best_val_loss = float('inf')
    patience = 20
    patience_counter = 0
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = evaluate(model, val_loader, criterion)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break
    
    return best_val_loss

In [85]:
models = {
    "Funnel": FunnelNetwork(),
    "Inverted Funnel": InvertedFunnelNetwork(),
    "Brick": BrickNetwork()
}

for name, model in models.items():
    best_val_loss = train_model(model, train_loader, val_loader)
    print(f"{name} Best Validation Loss: {best_val_loss:.4f}")

Funnel Best Validation Loss: 2507.8000
Inverted Funnel Best Validation Loss: 2519.5131
Brick Best Validation Loss: 2598.9504


In [ ]:
# Test case (don't edit this block)
assert True

---

Lecture: AI I - Advanced 

Next: [**Chapter 3.1: PyTorch Lightning**](../../03_advanced/01_lightning.ipynb)